In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/discord_chatbot/

/content/drive/MyDrive/discord_chatbot


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
import time
import tensorflow.compat.v1 as tf
import tensorflow_datasets as tfds
from tensorflow import keras
import keras as k
import sys
from sklearn.model_selection import train_test_split
import os

In [4]:
dataset = pd.read_csv('ChatbotData.csv')

In [5]:
dataset = dataset.drop('A', axis=1)
dataset = dataset.dropna()

In [6]:
X=dataset['Q']
y=dataset['label']

In [7]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

In [8]:
X_train.head()

6746                       숨어서 보기.
58                     감미로운 목소리 좋아
5155              혼자 있으니까 혼자 할게 많네
2865    아무리 열심히 해도 선천적인 건 못 따라가겠지?
3721                   이사하는 것도 일이다
Name: Q, dtype: object

In [9]:
print(max(len(each) for each in X_train))
print(max(len(each) for each in X_val))

56
51


In [10]:
X_train.shape

(8276,)

In [11]:
def insert_space(input):
  questions = []
  for sentence in input:
      sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
      sentence = sentence.strip()
      questions.append(sentence)
  return questions

In [12]:
X_train = insert_space(X_train)
X_val = insert_space(X_val)

In [13]:
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    X_train, target_vocab_size=2**13)

In [14]:

START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]

VOCAB_SIZE = tokenizer.vocab_size + 2

In [15]:

MAX_LENGTH = 128

def tokenize_and_filter(inputs):
  tokenized_inputs= []

  for sentence in inputs:
    sentence = START_TOKEN + tokenizer.encode(sentence) + END_TOKEN
    tokenized_inputs.append(sentence)


  tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_inputs, maxlen=MAX_LENGTH, padding='post')

  return tokenized_inputs

In [16]:
X_train = tokenize_and_filter(X_train)
X_val =tokenize_and_filter(X_val)

In [17]:
class TokenAndPositionEmbedding(tf.keras.layers.Layer):
    def __init__(self, max_len, vocab_size, embedding_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.pos_emb = tf.keras.layers.Embedding(max_len, embedding_dim)

    def call(self, x):
        max_len = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=max_len, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [18]:
max_len = 128
vocab_size = 20000

In [19]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, embedding_dim, num_heads=8):
        super(MultiHeadAttention, self).__init__()
        self.embedding_dim = embedding_dim # d_model
        self.num_heads = num_heads

        assert embedding_dim % self.num_heads == 0

        self.projection_dim = embedding_dim // num_heads
        self.query_dense = tf.keras.layers.Dense(embedding_dim)
        self.key_dense = tf.keras.layers.Dense(embedding_dim)
        self.value_dense = tf.keras.layers.Dense(embedding_dim)
        self.dense = tf.keras.layers.Dense(embedding_dim)

    def scaled_dot_product_attention(self, query, key, value):
        matmul_qk = tf.matmul(query, key, transpose_b=True)
        depth = tf.cast(tf.shape(key)[-1], tf.float32)
        logits = matmul_qk / tf.math.sqrt(depth)
        attention_weights = tf.nn.softmax(logits, axis=-1)
        output = tf.matmul(attention_weights, value)
        return output, attention_weights

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        # x.shape = [batch_size, seq_len, embedding_dim]
        batch_size = tf.shape(inputs)[0]

        # (batch_size, seq_len, embedding_dim)
        query = self.query_dense(inputs)
        key = self.key_dense(inputs)
        value = self.value_dense(inputs)

        # (batch_size, num_heads, seq_len, projection_dim)
        query = self.split_heads(query, batch_size)  
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)

        scaled_attention, _ = self.scaled_dot_product_attention(query, key, value)
        # (batch_size, seq_len, num_heads, projection_dim)
        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  

        # (batch_size, seq_len, embedding_dim)
        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.embedding_dim))
        outputs = self.dense(concat_attention)
        return outputs

In [20]:
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embedding_dim, num_heads, dff, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(embedding_dim, num_heads)
        self.ffn = tf.keras.Sequential(
            [tf.keras.layers.Dense(dff, activation="relu"),
             tf.keras.layers.Dense(embedding_dim),]
        )
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs) # 첫번째 서브층 : 멀티 헤드 어텐션
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output) # Add & Norm
        ffn_output = self.ffn(out1) # 두번째 서브층 : 포지션 와이즈 피드 포워드 신경망
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output) # Add & Norm

In [21]:
embedding_dim = 256  # embedding_dims
num_heads = 4  # attention head
dff = 128  # position wide ff

inputs = tf.keras.layers.Input(shape=(max_len,))
embedding_layer = TokenAndPositionEmbedding(max_len, vocab_size, embedding_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embedding_dim, num_heads, dff)
x = transformer_block(x)
x = tf.keras.layers.GlobalAveragePooling1D()(x)
x = tf.keras.layers.Dropout(0.1)(x)
x = tf.keras.layers.Dense(20, activation="relu")(x)
x = tf.keras.layers.Dropout(0.1)(x)
outputs = tf.keras.layers.Dense(3, activation="softmax")(x)



model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [22]:
model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])
history = model.fit(X_train, y_train, batch_size=32, epochs=10)

Epoch 1/10
259/259 [==============================] - 20s 61ms/step - loss: 1.0561 - accuracy: 0.4583
Epoch 2/10
259/259 [==============================] - 16s 61ms/step - loss: 0.7537 - accuracy: 0.6492
Epoch 3/10
259/259 [==============================] - 16s 60ms/step - loss: 0.4609 - accuracy: 0.8248
Epoch 4/10
259/259 [==============================] - 16s 61ms/step - loss: 0.2773 - accuracy: 0.9117
Epoch 5/10
259/259 [==============================] - 16s 61ms/step - loss: 0.1705 - accuracy: 0.9422
Epoch 6/10
259/259 [==============================] - 16s 60ms/step - loss: 0.1296 - accuracy: 0.9570
Epoch 7/10
259/259 [==============================] - 16s 61ms/step - loss: 0.0901 - accuracy: 0.9687
Epoch 8/10
259/259 [==============================] - 16s 60ms/step - loss: 0.0714 - accuracy: 0.9734
Epoch 9/10
259/259 [==============================] - 16s 61ms/step - loss: 0.0517 - accuracy: 0.9815
Epoch 10/10
259/259 [==============================] - 16s 61ms/step - loss: 0.054

In [23]:
loss, acc = model.evaluate(X_val,y_val)

111/111 [==============================] - 1s 8ms/step - loss: 1.6666 - accuracy: 0.7584


In [24]:
print(f'loss: {loss}, acc: {acc}')

loss: 1.6666259765625, acc: 0.7583873867988586


In [25]:
def evaluate(sentence):
  sentence = preprocess_sentence(sentence)

  sentence = tf.expand_dims(
      START_TOKEN + tokenizer.encode(sentence) + END_TOKEN, axis=0)
  
  sentence = tf.keras.preprocessing.sequence.pad_sequences(
      sentence, maxlen=MAX_LENGTH, padding='post')

  return model.predict(sentence)


In [26]:
def predict(sentence):

  sentence = preprocess_sentence(sentence)

  sentence = tf.expand_dims(
      START_TOKEN + tokenizer.encode(sentence) + END_TOKEN, axis=0)
  
  sentence = tf.keras.preprocessing.sequence.pad_sequences(
      sentence, maxlen=MAX_LENGTH, padding='post')
  print(sentence)

  return np.argmax(model.predict(sentence))

In [27]:
def preprocess_sentence(sentence):
  sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
  sentence = sentence.strip()
  return sentence

In [28]:
print(predict('오늘 우울하네요'))

[[6687   29 4393 3190 6688    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0]]
1


In [29]:
def make_directory(target_path):
  if not os.path.exists(target_path):
    os.mkdir(target_path)
    print('Directory ', target_path, ' Created ')
  else:
    print('Directory ', target_path, ' already exists')

SAVED_MODEL_PATH = './saved_model'
make_directory(SAVED_MODEL_PATH)
MODEL_DIR = SAVED_MODEL_PATH

version = 1
export_path = os.path.join(MODEL_DIR, str(version))
print('export_path = {}\n'.format(export_path))

tf.keras.models.save_model(
  model,
  export_path,
  overwrite=True,
  include_optimizer=True,
  save_format=None,
  signatures=None,
  options=None
)
print('\nSaved model:')

Directory  ./saved_model  already exists
export_path = ./saved_model/1



INFO:tensorflow:Assets written to: ./saved_model/1/assets


INFO:tensorflow:Assets written to: ./saved_model/1/assets



Saved model:


In [30]:
ltokenizer = tfds.deprecated.text.SubwordTextEncoder.load_from_file('tokenizer')

In [31]:
def encoding(sentence):

    sentence = preprocess_sentence(sentence)

    sentence = tf.expand_dims(
      START_TOKEN + tokenizer.encode(sentence) + END_TOKEN, axis=0)
  
    sentence = tf.keras.preprocessing.sequence.pad_sequences(
      sentence, maxlen=MAX_LENGTH, padding='post')

    return sentence